Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE        MCSE      ESS
alpha 154.59414050 0.273512673 0.00432461507 0.00435783953 1000
 beta   0.90539896 0.042831016 0.00067721782 0.00073036839 1000
sigma   5.10944010 0.194219316 0.00307087701 0.00265421513 1000

Quantiles:
          2.5%         25.0%       50.0%       75.0%       97.5%   
alpha 154.05300000 154.40800000 154.5965000 154.7822500 155.1360250
 beta   0.82116925   0.87702125   0.9041485   0.9333425   0.9913662
sigma   4.74036725   4.97741250   5.1051450   5.2300375   5.5101005



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.198 0.934151 5.29132; 155.086 0.812735 4.9899; … ; 154.633 0.883611 5.30566; 154.388 0.87005 5.1228]

Union{Missing, Float64}[154.435 0.842341 5.07821; 154.581 0.871619 5.07094; … ; 155.049 0.845639 5.22616; 154.148 0.904438 5.16832]

Union{Missing, Float64}[154.83 0.884846 5.19065; 154.568 0.921435 5.17732; … ; 154.321 0.930734 5.26147; 155.168 0.91389 4.95892]

Union{Missing, Float64}[154.499 0.812445 5.19045; 154.305 0.780548 5.14767; … ; 154.041 0.88512 5.41318; 154.662 0.875645 4.96161]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE        MCSE      ESS
alpha 154.59414050 0.273512673 0.00432461507 0.00435783953 1000
 beta   0.90539896 0.042831016 0.00067721782 0.00073036839 1000
sigma   5.10944010 0.194219316 0.00307087701 0.00265421513 1000

Quantiles:
          2.5%         25.0%       50.0%       75.0%       97.5%   
alpha 154.05300000 154.40800000 154.5965000 154.7822500 155.1360250
 beta   0.82116925   0.87702125   0.9041485   0.9333425   0.9913662
sigma   4.74036725   4.97741250   5.1051450   5.2300375   5.5101005



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*